## Utilitary functions

In [ ]:
import numpy as np

def make_grid(s,e):
  """
  Creating a square grid
  """
  x = np.linspace(s, e, 500)
  y = np.linspace(s, e, 500)
  X,Y = np.meshgrid(x,y)
  pos = np.empty(X.shape + (2,))
  pos[:, :, 0] = X; pos[:, :, 1] = Y

  return X, Y, pos

def ptlist_to_xy(l):
  """
  Converting a list of points to lists x,y
  """
  return list(zip(*l))

## 1.1 Preparatory work

### Plotting the density function of the mixture

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import multivariate_normal
from mpl_toolkits.mplot3d import Axes3D

# Mixture parameters
mu_1 = [-3, 0]
sigma_1 = [[5, -2], [-2, 1]]
pi_1 = 0.3

mu_2 = [3, 0]
sigma_2 = [[5, 2], [2, 2]]
pi_2 = 0.7

# Creating a grid
X,Y,pos = make_grid(-10, 10)

# Building a mixture
rv_1 = multivariate_normal(mu_1, sigma_1)
rv_2 = multivariate_normal(mu_2, sigma_2)

# Make a 3D plot
fig = plt.figure(num=None, figsize=(20, 13), facecolor='w', edgecolor='k')
ax = fig.gca(projection='3d')
ax.plot_surface(X, Y, pi_1*rv_1.pdf(pos) + pi_2*rv_2.pdf(pos), cmap='viridis', linewidth=0)
plt.show()


### Simulating a sample of size 500

In [ ]:
N_POINTS = 500

# Sampling the categories
k_s = np.random.choice([1,2], N_POINTS, p=[pi_1, pi_2])

# Sampling from the gaussians
g_1 = np.count_nonzero(k_s == 1)
pts_g1 = np.random.multivariate_normal(mu_1, sigma_1, g_1).T
pts_g2 = np.random.multivariate_normal(mu_2, sigma_2, N_POINTS - g_1).T

# Make a 2D plot
fig = plt.figure(num=None, figsize=(10, 7), facecolor='w', edgecolor='k')
ax = fig.gca()

# Plotting points from both Gaussians
plt.scatter(*pts_g1)
plt.scatter(*pts_g2)
plt.show()

In [ ]:
a = np.random.choice([1,2], 500, p=[pi_1, pi_2])

### Processing the Unistroke dataset

In [ ]:
# Unzipping
%%capture
![[ -d Unistroke ]] || unzip unistroke.zip

In [ ]:
from glob import glob

def load_data():
  """
  Loading the Unistroke dataset
  """
  files = glob('./Unistroke/A*.txt')
  data = [open(f, "r").read() for f in files]
  return [d.split('\n') for d in data]

# Converting data to a list of vectors
flatten = lambda l: [item for sublist in l for item in sublist]
pts = load_data()
pts = [e for e in flatten(pts) if e]
pts = [v.split('\t') for v in pts]
pts = [[float(n) for n in l] for l in pts]

pair_pts = pts
pts = ptlist_to_xy(pts)

# Plotting the data
fig = plt.figure(num=None, figsize=(12, 7), facecolor='w', edgecolor='k')
plt.gca().invert_yaxis()
plt.scatter(*pts)
plt.show()

### Building a normalized list of vectors

In [ ]:
# List of points, per file
pts_list = [[v.split('\t') for v in l] for l in load_data()]
pts_list = [[[float(v) for v in e] for e in l if e[0]] for l in pts_list]

# Computing x[t+1] - x[t]
dirs = []
for l in pts_list:
  for x0,x1 in zip(l,l[1:]):
    x1, x0 = np.asarray(x1), np.asarray(x0)
    norm = np.linalg.norm(x1 - x0)
    if norm: # Filtering 0 values
      dirs.append((x1 - x0)/norm)

# Plotting the result
fig = plt.figure(num=None, figsize=(7, 7), facecolor='w', edgecolor='k')
for x,y in dirs:
    plt.plot(x, y, "g.")
plt.show()

## 1.2 Data analysis: Gaussian model

### Bivariate GMM estimation: 2 components

In [ ]:
import numpy as np
np.random.seed(1)

In [ ]:
from sklearn import mixture

# Fitting the mixture model
g = mixture.GaussianMixture(n_components=2)
g.fit(dirs)
mus, covs, pis = g.means_, g.covariances_, g.weights_
save_g = g

# Predicting the component for each point
k_s = g.predict(dirs)

# Predicting probabilities
p_s = g.predict_proba(dirs)

In [ ]:
from matplotlib import ticker

def plot_gmm(mus, covs, pis, k_s, cols, dim, ticks):
  """
  Plotting a Gaussian Mixture Model
  """
  def plot_mixture():
    X,Y,pos = make_grid(-1.15,1.15)

    # Building the mixture
    pdf = None
    for mu,cov,pi in zip(mus,covs,pis):
      if pdf is None:
        pdf = pi*multivariate_normal(mu, cov).pdf(pos)
      else:
        pdf += pi*multivariate_normal(mu, cov).pdf(pos)

    ax = plt.gca()
    CS = ax.contour(X, Y, pdf,
                locator=ticker.LogLocator(base=10, numticks=18),
                cmap='Greys')
    ax.clabel(CS, inline=1, fmt='%.e', fontsize=10)
    CB = plt.colorbar(CS)
    return ax

  fig = plt.figure(num=None, figsize=dim, facecolor='w', edgecolor='k')

  # Plotting normalized vectors
  for (x,y),k in zip(dirs,k_s):
      plt.plot(x, y, cols[k], marker='.')

  # Plotting mixture
  ax = plot_mixture()

  plt.show()

plot_gmm(mus, covs, pis, k_s, ['lime', 'slateblue'], (12,10), 15)

### Bivariate GMM estimation: 3,4 components

In [ ]:
# Fitting the data with an additional component
g = mixture.GaussianMixture(n_components=3)
g.fit(dirs)
mus, covs, pis = g.means_, g.covariances_, g.weights_

# Predicting the component for each point
k_s = g.predict(dirs)
plot_gmm(mus, covs, pis, k_s, ['lime', 'slateblue', 'red'], (10,10), 15)

In [ ]:
# Fitting the data with 4 components
g = mixture.GaussianMixture(n_components=4)
g.fit(dirs)
mus, covs, pis = g.means_, g.covariances_, g.weights_

# Predicting the component for each point
k_s = g.predict(dirs)
plot_gmm(mus, covs, pis, k_s, ['lime', 'slateblue', 'red', 'yellow'], (10,10), 15)

### Plotting posterior probabilities (for $K=2$)

In [ ]:
# Recovering the previous GMM
mus, covs, pis = save_g.means_, save_g.covariances_, save_g.weights_

# Predicting probabilities
p_s = save_g.predict_proba(dirs)

In [ ]:
from matplotlib import ticker,colors,cm

def plot_probas(p_s, dim, ticks):
  """
  Plotting conditional probabilities
  """
  fig = plt.figure(num=None, figsize=dim, facecolor='w', edgecolor='k')

  min_p = 1e-15
  max_p = max(p_s)

  # Plotting normalized vectors
  cmap = plt.cm.viridis
  norm = colors.LogNorm(vmin=min_p, vmax=max_p)
  for (x,y),p in zip(dirs,p_s):
      plt.plot(x, y, color=cmap(norm(p)), marker='o', ms=10, alpha=0.85)
  fig.colorbar(cm.ScalarMappable(norm=norm, cmap=cmap),
               ax=plt.gca(), aspect=30)

  plt.show()

plot_probas(p_s[:,1], (10,8), 15)

### Plotting marginal histograms

In [ ]:
# Extracting lists of x and y coordinates
xs, ys = ptlist_to_xy(dirs)
xs, ys = np.array(xs), np.array(ys)

In [ ]:
from scipy.stats import norm

def plot_comp(data, legend, range, bins, figsize):
  """
  Plotting a marginal histogram
  """
  fig = plt.figure(num=None, figsize=figsize, facecolor='w', edgecolor='k')

  # Plotting histogram
  axes = plt.gca()
  axes.set_xlim(range)

  plt.hist(data, bins=bins, density=True, color='darkgrey')
  plt.margins(0)

  plt.xlabel(legend)
  plt.ylabel('Frequency')
  plt.grid(False)

def plot_gmm_1d(mus, covs, pis, range):
  """
  Plotting a one-dimensional Gaussian Mixture Model
  """
  t = np.arange(*range, 0.005)

  # Building the mixture
  pdf = None
  for mu,cov,pi in zip(mus,covs,pis):
    if pdf is None:
      pdf = pi*norm(mu, cov).pdf(t)
    else:
      pdf += pi*norm(mu, cov).pdf(t)

  pdf = pdf.reshape(-1)
  plt.plot(t, pdf, linewidth=2.5, label="$\Sigma$", color='red', zorder=10)
  return ax

def plot_1D_fit(data, legend, range=[-1., 1.], bins=150, figsize=(10,5)):
  """
  Plotting the marginal fit
  """
  # Plotting X/Y histogram
  plot_comp(data, legend, range, bins, figsize)

  # Fitting a univariate Gaussian mixture
  g = mixture.GaussianMixture(n_components=2)
  g.fit(data.reshape(-1,1))
  mus, covs, pis = g.means_, g.covariances_, g.weights_

  # Predicting the component for each point
  plot_gmm_1d(mus, covs, pis, range)

In [ ]:
# Plotting X histogram
plot_1D_fit(xs, "Value of the $x$-component")

In [ ]:
# Plotting Y histogram
plot_1D_fit(ys, "Value of the $y$-component")

### Plotting each cluster separately

#### Clusters in X

In [ ]:
plot_1D_fit(xs, "Value of the $x$-component", [-0.7, -0.2], 100, (7,5))

In [ ]:
plot_1D_fit(xs, "Value of the $x$-component", [0.8, 1], 100, (7,5))

#### Clusters in Y

In [ ]:
plot_1D_fit(ys, "Value of the $y$-component", [-1., -0.4], 100, (7,5))

In [ ]:
plot_1D_fit(ys, "Value of the $y$-component", [0.85, 1.], 100, (7,5))

## 2.0 Mandatory additional questions


### Transformation into angular data


In [ ]:
def plot_angular_data(angles, density=True):
  """
  Plotting angular data as a histogram.
  """
  fig = plt.figure(num=None, figsize=(10,6), facecolor='w', edgecolor='k')
  plt.hist(angles, bins=np.arange(-3,4, 0.2)-0.5,
           color='lightgray', edgecolor='gray',
           density=density)
  plt.xlabel('Angle (rad)')

def p_to_angle(x,y):
  """
  Converting a point to an angle.
  """
  if x == 0:
    angle = (-np.sign(y)*np.pi/2)
  elif x < 0:
    angle = (-np.sign(y)*np.pi - np.arctan(y/x))
  else:
    angle = (-np.arctan(y/x))
  return angle

angles = [p_to_angle(x,y) for x,y in dirs]

In [ ]:
plot_angular_data(angles, density=False)

### Mixture of Von Mises : 2 components


#### Expectation Maximization for Von Mises Mixture

In [ ]:
import numpy as np
from scipy import optimize
from scipy.special import iv


def I0(kappa):
  """
  Modified Bessel function of order 0
  """
  return iv(0,kappa)

def I1(kappa):
  """
  Modified Bessel function of order 1
  """
  return iv(1,kappa)

def von_mises_pdf(x_n, mu_k, kappa_k):
  return np.exp(kappa_k*np.cos(x_n - mu_k))/(2*np.pi*I0(kappa_k))

def expectation_eta(x, pi_l, mu_l, kappa_l):
  """
  Computing the responsibilities
  """
  eta = [[_ for j in range(len(pi_l))] for _ in range(len(x))]
  for i in range(len(x)):
    sum = 0
    for pi,mu,kappa in zip(pi_l,mu_l,kappa_l):
      sum += pi*von_mises_pdf(x[i], mu, kappa)
    for j,(pi,mu,kappa) in enumerate(zip(pi_l,mu_l,kappa_l)):
      eta[i][j] = pi*von_mises_pdf(x[i], mu, kappa)/sum
  return eta

def update_pi(eta, k):
  """
  Reestimating pi
  """
  pi = 0
  for i in range(len(eta)):
    pi += eta[i][k]
  return pi/len(eta)

def update_mu(eta, x, k):
  """
  Reestimating mu
  """
  sum1,sum2 = 0,0
  for i in range(len(x)):
    sum1 += eta[i][k]*np.sin(x[i])
    sum2 += eta[i][k]*np.cos(x[i])
  return np.arctan2(sum1,sum2)

def update_kappa(eta, x, k, mu_star, kappa):
  """
  Reestimating kappa
  """
  sum1,sum2 = 0,0
  for i in range(len(x)):
    sum1 += eta[i][k]*np.cos(x[i] - mu_star[k])
    sum2 += eta[i][k]
  sol = lambda kappa: I1(kappa)/I0(kappa) - sum1/sum2
  return optimize.fsolve(sol, kappa[k])

In [ ]:
def init_params(N):
  """
  Initializing mixtures parameters
  """
  pi = [1./N for _ in range(N)]
  mu = np.random.uniform(-np.pi, np.pi, N)
  kappa = [10 for _ in range(N)]
  return pi, mu, kappa

def EM_von_mises(x, N, max_iterations):
  """
  Expectation Maximization for Von Mises Mixture
  """
  pi, mu, kappa = init_params(N)

  iter = 0
  K = len(mu)
  while iter < max_iterations:
    # E-step
    eta = expectation_eta(x, pi, mu, kappa)
    # M-step
    pi = [update_pi(eta, k) for k in range(K)]
    mu = [update_mu(eta, x, k) for k in range(K)]
    kappa = [update_kappa(eta, x, k, mu, kappa) for k in range(K)]
    iter += 1

  # Computing arg max_k (eta_k) for every sample
  eta = expectation_eta(x, pi, mu, kappa)
  k_s = np.argmax(eta, axis=1).flatten()

  return pi, mu, kappa, k_s

#### Plotting 3-components von Mises fit

In [ ]:
def von_mises_mix_pdf(x, pi_l, mu_l, kappa_l):
  """
  Computing a Von Mises Mixture PDF.
  """
  pdf = None
  for pi,mu,kappa in zip(pi_l, mu_l, kappa_l):
    if pdf is None:
      pdf = pi*von_mises_pdf(x, mu, kappa)
    else:
      pdf += pi*von_mises_pdf(x, mu, kappa)
  return pdf

def plot_vm_mix(pi_l, mu_l, kappa_l, color='red', alpha=1.0):
  """
  Plotting a Von Mises mixture
  """
  t = np.arange(*[-3.2,3.2], 0.005)

  # Building the mixture
  pdf = von_mises_mix_pdf(t, pi_l, mu_l, kappa_l)
  pdf = pdf.reshape(-1)
  plt.plot(t, pdf, linewidth=2.5,
           color=color, zorder=10, alpha=alpha)
  return ax

# Running EM
pi_l, mu_l, kappa_l, k_s = EM_von_mises(angles, 3, 100)

# Plotting the histogram
plot_angular_data(angles)

# Plotting the mixture
plot_vm_mix(pi_l, mu_l, np.array(kappa_l))

In [ ]:
mu_l, pi_l, kappa_l

#### Plotting the Von Mises fit on the unit circle

In [ ]:
def von_mises_mix_pdf(x, pi_l, mu_l, kappa_l):
  """
  Computing a Von Mises Mixture PDF.
  """
  pdf = None
  for pi,mu,kappa in zip(pi_l, mu_l, kappa_l):
    if pdf is None:
      pdf = pi*von_mises_pdf(x, mu, kappa)
    else:
      pdf += pi*von_mises_pdf(x, mu, kappa)
  return pdf

def von_mises_ct(pi_l, mu_l, kappa_l):
  def von_mises_clj(x,y):
    return von_mises_mix_pdf(p_to_angle(x,y),
                             pi_l, mu_l, kappa_l)
  return von_mises_clj

def plot_vm_contours(pi_l, mu_l, kappa_l, angles, k_s, cols):
  """
  Plotting angles on the unit circle,
  with the Von Mises PDF contours.
  """
  # Plotting angles
  fig = plt.figure(num=None, figsize=(12,10), facecolor='w', edgecolor='k')
  for t,k in zip(angles, k_s):
    x,y = np.cos(t), np.sin(t)
    plt.plot(x, y, cols[k-1], marker='.')

  # Plotting mixture contour levels
  X,Y,p = make_grid(-1.2, 1.2)

  pdf = von_mises_ct(pi_l, mu_l, kappa_l)
  pdf_ct = [[pdf(x,y) for (x,y) in p[c]] for c in range(np.shape(p)[0])]
  pdf_ct = np.array(pdf_ct).squeeze()

  ax = plt.gca()
  CS = ax.contour(X, Y, pdf_ct,
                  locator=ticker.LogLocator(base=10, numticks=18),
                  cmap='Greys')
  ax.clabel(CS, inline=1, fmt='%.e', fontsize=10)
  CB = plt.colorbar(CS)

In [ ]:
# Plotting the graph
plot_vm_contours(pi_l, mu_l, kappa_l, angles, k_s, ['lime', 'slateblue', 'red'])

### Plotting a 2-components Von-Mises mixture

In [ ]:
pi_l, mu_l, kappa_l, k_s = EM_von_mises(angles, 2, 50)

# Plotting the graph
plot_vm_contours(pi_l, mu_l, kappa_l, angles, k_s, ['lime', 'slateblue', 'red'])

## 3.0 Optional additional questions

### Consistency of an estimator

In [ ]:
import numpy as np
from sklearn import mixture


def sample_gmm(n, pi_l, mu_l, sigma_l):
  """
  Sampling from a GMM with given parameters.
  """
  # Sampling the categories
  n_comp = len(pi_l)
  k_s = np.random.choice(list(range(1,n_comp+1)), size=n, p=pi_l)

  # Sampling from each component
  samples = []
  for k, (mu,sigma) in enumerate(zip(mu_l,sigma_l)):
    g_k = np.count_nonzero(k_s == (k+1))
    gen = np.random.multivariate_normal(mu, sigma, g_k).T
    samples += list(zip(*gen))

  return np.array(samples)

def BIC_estimator(samples, theta):
  """
  Estimate the optimal number of components
  of mixture-generated data "samples"
  by exploring the parameter space "theta",
  using the bayesian information criterion.
  """
  best_bic = np.infty
  best_cmp = -1

  for n_comp in theta:
    gmm = mixture.GaussianMixture(n_components=n_comp)
    gmm.fit(samples)
    
    # Computing BIC
    gmm_bic = gmm.bic(samples)

    if gmm_bic < best_bic:
      best_bic = gmm_bic
      best_cmp = n_comp

  return best_cmp

def AIC_estimator(samples, theta):
  """
  Estimate the optimal number of components
  of mixture-generated data "samples"
  by exploring the parameter space "theta",
  using the Akaike information criterion.
  """
  best_aic = np.infty
  best_cmp = -1

  for n_comp in theta:
    gmm = mixture.GaussianMixture(n_components=n_comp)
    gmm.fit(samples)
    
    # Computing AIC
    gmm_aic = gmm.aic(samples)

    if gmm_aic < best_aic:
      best_aic = gmm_aic
      best_cmp = n_comp

  return best_cmp


# Fixing the 4-components GMM's parameters
pi_l = [0.25, 0.45, 0.15, 0.15]
mu_l = [[-3, 0], [3, 0], [8,1], [-8,1]]
sigma_l = [[[5, -2], [-2, 1]], [[5, 2], [2, 2]],
           [[7, 2], [2, 3]],   [[3, 2], [2, 7]]]

In [ ]:
# Initializing the parameter space to explore
MAX_K = 20
theta = list(range(1, MAX_K+1)) # Here we picked 1..20

# Samples with increasing size
n_values = list(range(MAX_K, 750, 5))

est_n_comp = []
est_n_comp_aic = []
for N in n_values:
  # Generating N datapoints from a 4-components GMM
  X = sample_gmm(N, pi_l, mu_l, sigma_l)

  # Computing the BIC-estimated number of components
  est_n_comp += [BIC_estimator(X, theta)]
  est_n_comp_aic += [AIC_estimator(X, theta)]

#### Plotting the results

In [ ]:
from matplotlib import pyplot as plt

fig = plt.figure(num=None, figsize=(12, 6), facecolor='w', edgecolor='k')
plt.plot(n_values, est_n_comp, linewidth=2.5, color='blue', label="BIC-estimated $K$")
plt.plot(n_values, est_n_comp_aic, linewidth=2.5, color='red', label="AIC-estimated $K$")

plt.axhline(4, color='green', label="Real $K$")
plt.legend(loc="upper right")
plt.yticks(np.arange(0, MAX_K+1, 5))

plt.xlabel("Number of observations $N$")
plt.ylabel("Value of $\widehat{K}$")

plt.show()

### Implementing Von Mises sampling and Von Mises mixtures

In [ ]:
import numpy as np
from scipy import stats
from scipy.special import iv


def I0(kappa):
  """
  Modified Bessel function of order 0
  """
  return iv(0,kappa)

def I1(kappa):
  """
  Modified Bessel function of order 1
  """
  return iv(1,kappa)

def von_mises_pdf(x, mu, kappa):
  """
  Von Mises PDF function
  """
  return np.exp(kappa*np.cos(x - mu))/(2*np.pi*I0(kappa))

def von_mises_sample(mu, kappa, n):
  """
  Sampling from a Von Mises distribution

    Reference:
      [1] Barabesi, L. (1995) Generating von Mises variates by
      the ratio-of-uniforms method, Statistica Applicata 7, 417-426.
  """
  samples = [0 for _ in range(n)]

  for k in range(n):
    s = 1/np.sqrt(kappa) if kappa > 1.3 else np.pi*np.exp(-kappa)

    while True:
      r1,r2 = np.random.uniform(0, 1, 2)
      theta = s*(2*r2-1)/r1

      if np.abs(theta) > np.pi:
        continue
      if kappa*(theta*theta) < 4 - 4*r1:
        break
      if kappa*np.cos(theta) < 2*np.log(r1) + kappa:
        continue
      break
    samples[k] = theta
  return np.array(samples) + np.array([mu]*n)

In [ ]:
# Test parameters
kappa = 4
mu = 0

# Instantiating Von-Mises CDF
vm_fun = stats.vonmises(kappa=kappa, loc=mu)

# Sampling 100k points
points = von_mises_sample(mu, kappa, 100000)

# Plotting the histogram
plot_angular_data(points)

# Plotting sampled points
t = np.arange(*[-3.2,3.2], 0.005)
plt.plot(t, vm_fun.pdf(t), linewidth=2.5, color='red', zorder=10)
plt.xlabel('Angle (rad)')

In [ ]:
# Kolmogorov-Smirnov test
from scipy import stats

def kolmogorov_smirnov(kappa):
  """
  Performing a Kolmogorov-Smirnov statistical test
  to compare the quality of our sampling function.
  """
  vm_fun = stats.vonmises(kappa=kappa, loc=0)
  t = stats.kstest(von_mises_sample(0, kappa, 100000), vm_fun.cdf)
  print(t)

kolmogorov_smirnov(0.5)
kolmogorov_smirnov(10)
kolmogorov_smirnov(50)
kolmogorov_smirnov(100)

In [ ]:
def von_mises_mixture_sample(pi_l, mu_l, kappa_l, n):
  """
  Sampling from a Von Mises mixture.
  """
  # Sampling the categories
  n_comp = len(pi_l)
  k_s = np.random.choice(list(range(1,n_comp+1)), size=n, p=pi_l)

  # Sampling from each component
  classes = []
  samples = []
  for k, (mu,kappa) in enumerate(zip(mu_l,kappa_l)):
    g_k = np.count_nonzero(k_s == (k+1))
    samples += list(von_mises_sample(mu, kappa, g_k))
    classes += g_k*[k]

  return samples, classes

# Sampling 1000 points from a mixture
kappa_l = [15, 10, 60]
mu_l = [(-2/3)*np.pi, 0, (2/3)*np.pi]
pi_l = [0.3, 0.25, 0.45]

angles,k = von_mises_mixture_sample(pi_l, mu_l, kappa_l, 10000)

In [ ]:
# Plotting the resulting histogram
plot_angular_data(angles)

# Plotting the mixture
plot_vm_mix(pi_l, mu_l, np.array(kappa_l))

#### Re-estimating Von Mises parameters

In [ ]:
pi_p, mu_p, kappa_p, k_s = EM_von_mises(angles, 3, 10)
plot_vm_contours(pi_p, mu_p, kappa_p, angles, k, ['lime', 'slateblue', 'red'])

In [ ]:
# Plotting the resulting histogram
plot_angular_data(angles)

# Plotting the mixture
plot_vm_mix(pi_l, mu_l, np.array(kappa_l), color="green", alpha=0.5)
plot_vm_mix(pi_p, mu_p, np.array(kappa_p), alpha=0.5)

In [ ]:
def error_p(a, r_a, b, r_b, c, r_c):
  """
  Computing error percentages of input parameters.
  """
  def error(n0, n1):
    return np.abs(((n0 - n1)/n1)*100)

  p_1 = error(a, r_a)
  p_2 = error(b, r_b)
  p_3 = error(c, r_c)

  return np.mean([p_1, p_2, p_3])